# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [1]:
import pandas as pd
import json
from collections import defaultdict,OrderedDict
import sorting
import re
from datetime import datetime, timedelta
import time
import xml.etree.ElementTree as ET  

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [2]:
def myFunc(e):
    return e['timestamp']

datos = json.load(open('trazasOrdenadas.json'))

# Buscar si es nivel principal

In [3]:
def nivel_principal(nivel):
    erAnomalyLevels = re.compile(r'\btutorials|editor_level|tip|procedures')
    return not erAnomalyLevels.search(nivel)

# Obtener tarjetas de fin de nivel

In [14]:
def dict_factory():
    return defaultdict(dict_factory)

def obtener_codigo(datos):
    #Recorre el array y guarda los bloques de tarjetas de los niveles completados
    #Las tarjetas se manipulan cuando:
    
    erLevel = re.compile(r'\blevel$\b')
    erButton = re.compile(r'button\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erProgressed = re.compile(r'\bprogressed$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erStartButton = re.compile(r'\bstart_button$\b')
    erLevelExit = re.compile(r'\blevel_exit_button$\b')
    erIdLevel = re.compile(r'/')
    tarjetas = defaultdict(defaultdict)
    intentos_fallidos = defaultdict(defaultdict)
    progresados = defaultdict(defaultdict)
    usuarioAnt = ''
    encontradoInt = False
    encontradoProg = False
    for evento in datos:
        obj = evento["object"]["definition"]["type"]
        objectId = evento["object"]["id"]
        verb = evento["verb"]["id"]
        name = evento["actor"]["name"]
        levelCode = erIdLevel.split(objectId)[-1]
        #Buscamos aquellos niveles que sean principales
        if nivel_principal(objectId):
            if erCompleted.search(verb):
                #Hemos encontrado un nivel completado de un codigo que teniamos guardado
                if levelCode in progresados and name in progresados[levelCode]:
                    if evento["result"]["success"] == True:
                        #Nivel completado con éxito
                        if name not in tarjetas[levelCode]:
                            tarjetas[levelCode][name] = [progresados[levelCode][name]]
                        else:
                            tarjetas[levelCode][name].append(progresados[levelCode][name])
                    else: 
                        #Intento fallido
                        if name not in intentos_fallidos[levelCode]:
                            intentos_fallidos[levelCode][name] = [progresados[levelCode][name]]
                        else:
                            intentos_fallidos[levelCode][name].append(progresados[levelCode][name])
                    encontradoProg = False
                    usuarioAnt = ''               
                    

            elif "result" in evento:
                if "extensions" in evento["result"]:
                    if "code" in evento["result"]["extensions"]:
                        #Si hay codigo buscamos que el codigo sea final
                        if erCompleted.search(verb):
                            #Guarda el codigo de los intentos fallidos
                            if name not in intentos_fallidos[levelCode]:
                                intentos_fallidos[levelCode][name] = [evento["result"]["extensions"]["code"]]
                            else:
                                intentos_fallidos[levelCode][name].append(evento["result"]["extensions"]["code"])
                                

                        elif erProgressed.search(verb):
                            #Comprobamos que todos los elementos que entran aquí son de codigo de niveles completados y no fallidos
                            progresados[levelCode][name] = evento["result"]["extensions"]["code"]


    return tarjetas


listaCodigo = obtener_codigo(datos)



## Obtenemos las tarjetas usadas durante un nivel

In [ ]:




def str_code_to_xml(code):
    parsed = ET.fromstring(code)
    return parsed

def obtener_tarjetas(datos):
    #Recorre el array y guarda los bloques de tarjetas de los niveles completados
    #Las tarjetas se manipulan cuando:
    
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erStartButton = re.compile(r'\bstart_button$\b')
    erLevelExit = re.compile(r'\blevel_exit_button$\b')
    erIdLevel = re.compile(r'/')
    niveles_completados = defaultdict()
    antId = ''
    antLevel = ''
    infoBloque = ""
    encontrado = False
    for evento in datos:
        obj = evento["object"]["definition"]["type"]
        objectId = evento["object"]["id"]
        verb = evento["verb"]["id"]
        name = evento["actor"]["name"]
        if erLevel.search(obj):
            levelCode = erIdLevel.split(objectId)[-1]
            if erCompleted.search(verb):
                if evento["result"]["success"] and nivel_principal(objectId):
                    niveles_completados[levelCode].append({"name":name,"timestamp": evento["timestamp"]})
                    encontrado = False
        
        

    return niveles_completados


niv_completados = obtener_tarjetas(datos)
pd.DataFrame(niv_completados)

ValueError: All arrays must be of the same length